# agex: Runtime Interop

**The Story:** We, as the developer, will act as the orchestrator. Our goal is to create a sales trend visualization. We will use two specialist AI agents:
1. A `DatabaseExpert` that can query a database and return a `pandas.DataFrame`.
2. A `VisualizationExpert` that can take a `DataFrame` and create a `plotly.Figure`.

Notice what we **don't** have to do: no writing JSON schemas, no saving data to files to pass between steps, and no context window explosions. Let's begin.


### Setup and Environment

First, let's import our libraries and create an in-memory SQLite database populated with some realistic, seasonal sales data. The `create_in_memory_db` function returns a live `sqlite3.Connection` object.


In [ ]:
import pandas as pd
import plotly.express as px
import sqlite3
from IPython.display import display
from plotly.graph_objects import Figure
from agex import Agent, Versioned, events
from agex.helpers import register_pandas 
from examples.db_primer import PRIMER

# Our helpers for demo setup
from examples.notebook.helper import create_in_memory_db

db_connection = create_in_memory_db()
state = Versioned()  # we'll capture the agents' state as they work

### Define the DatabaseExpert

Next, we'll define our `DatabaseExpert`. We'll give it access to the live `db_connection` object and the `pandas` library. 

In [2]:

db_expert = Agent(
    name="db_expert", 
    primer=PRIMER, # coach the agent on how to use the db
)

# Give the agent access to the live database connection
db_expert.module(
    db_connection,  # we register instance methods just like we do for module fns
    name="db",  # name is required when registering instance methods
    include=["execute", "executemany", "commit", "cursor"],
)

# also register the Cursor class for gathering results
db_expert.cls(sqlite3.Cursor, include=["fetchone", "fetchall", "fetchmany", "description"])

# use the pre-existing helper to register pandas
register_pandas(db_expert)

# by taking an inital action for the agent, we can give it a head start,
# otherwise we'd spend an extra LLM call deciding to explore the schema
SETUP_ACTION = """
# find columns in the sales table
columns_info = db.execute("PRAGMA table_info(sales)").fetchall()
columns = [col[1] for col in columns_info]

# find distinct product names
distinct_products = db.execute("SELECT DISTINCT product_name FROM sales").fetchall()
product_names = [row[0] for row in distinct_products]

task_continue("Columns in 'sales' table:", columns, "Distinct product_names:", product_names)
"""

@db_expert.task(setup=SETUP_ACTION)
def get_data(prompt: str) -> pd.DataFrame: # type: ignore[return-value]
    """Runs a SQL query and returns the result as a pandas DataFrame."""
    pass

# Let's ask the agent to get the sales data for umbrellas.
sales_df = get_data("Please give me umbrella sales by day", state=state)

### Inspect the Data

This is the first **'wow' moment**. The `get_data` call didn't return a string or JSON. It returned a live, rich `pandas.DataFrame` object directly into our session. The agent came to the data, performed its work, and handed back a real object.

Let's prove it by displaying the DataFrame:


In [3]:
display(sales_df)

,date,total_sales
0,2022-01-01,45
1,2022-01-02,105
2,2022-01-03,120
3,2022-01-04,120
4,2022-01-05,120
...,...,...
725,2023-12-27,15
726,2023-12-28,0
727,2023-12-29,0
728,2023-12-30,0


### Step 4: Define and Use the VisualizationExpert

Now we'll define our second specialist. Its job is to take a DataFrame and create a plot. We will pass the `sales_df` object we just received directly to this new agent.


In [4]:
viz_expert = Agent(
    name="viz_expert",
    primer="You are an expert at creating plots from pandas DataFrames using plotly express."
)

# Give the agent access to the plotly express library
viz_expert.module(px)
register_pandas(viz_expert)

@viz_expert.task
def plot_dataframe(prompt: str, df: pd.DataFrame) -> Figure: # type: ignore[return-value]
    """Create a plot from the given data according to the prompt."""
    pass

# Let's ask the agent to plot the DataFrame we got from the first agent.
sales_plot = plot_dataframe("Plot daily sales smoothed by a 30-day moving average", sales_df, state=state)

### Step 5: Inspect the Final Artifact

And here is the second **'wow' moment**. We passed a `DataFrame` object from one agent to another, and the second agent returned a fully interactive `plotly.Figure` object. 

Let's display it:


In [5]:
display(sales_plot)

### Conclusion: The `agex` Difference

We just orchestrated a complex workflow involving a database, a large dataset, and a visualization library, all using natural language prompts.

Crucially, we leveraged the powerful abstractions we *already* use as developers (`pandas.DataFrame`, `plotly.Figure`) instead of being forced into a restrictive tool-calling paradigm. The agents worked with our existing code and objects natively.

This is the core advantage of `agex`: it respects your code and enables a more powerful, flexible, and Pythonic way to build agentic systems.

---

But wait... how about an encore? Remember the `state` we created at the beginning? It's captured our both what our agents did, and their workspaces over time. Let's see just how they tackled these tasks.

In [6]:
for evt in events(state):
    display(evt)

TaskStartEvent(TaskStartEvent[db_expert] @ 05:01:00)

ActionEvent(ActionEvent[db_expert] @ 05:01:00)

OutputEvent(OutputEvent[db_expert] @ 05:01:00)

ActionEvent(ActionEvent[db_expert] @ 05:01:04)

,date,total_sales
0,2022-01-01,45
1,2022-01-02,105
2,2022-01-03,120
3,2022-01-04,120
4,2022-01-05,120
...,...,...
725,2023-12-27,15
726,2023-12-28,0
727,2023-12-29,0
728,2023-12-30,0


ActionEvent(ActionEvent[db_expert] @ 05:01:06)

,date,total_sales
0,2022-01-01,45
1,2022-01-02,105
2,2022-01-03,120
3,2022-01-04,120
4,2022-01-05,120
...,...,...
725,2023-12-27,15
726,2023-12-28,0
727,2023-12-29,0
728,2023-12-30,0


,date,total_sales
0,2022-01-01,45
1,2022-01-02,105
2,2022-01-03,120
3,2022-01-04,120
4,2022-01-05,120
...,...,...
725,2023-12-27,15
726,2023-12-28,0
727,2023-12-29,0
728,2023-12-30,0


ActionEvent(ActionEvent[viz_expert] @ 05:01:13)

SuccessEvent(SuccessEvent[viz_expert] @ 05:01:13)